In [1]:
## Import for data generation
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [190]:
ID_21659081 = np.load("C:\\Users\\nico\\Desktop\\Stanford\\OneDrive - Stanford\\Courses\\CS229\\finalproject\\data\\United_States_Fires\\United_States_2019_Fires\\apr\\multi_day\\21659081.npy")

In [208]:
training_set = ID_21659081
x_train = []
y_train = []

n_future = 1
n_past = 1

for i in range(0 , len(training_set) - n_future - n_past + 1):
    x_train.append(training_set[i : i + n_past])
    y_train.append(training_set[i + n_past: i + n_past + n_future])

x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[1], x_train.shape[0], x_train.shape[2], x_train.shape[3]))

y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[2], y_train.shape[3]))



In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import tensorflow as tf

In [6]:
from pyspark import SparkContext
sc =SparkContext()

In [209]:
regressor = Sequential()
print(x_train.shape)
dropout = 0.9
regressor.add(tf.keras.layers.Bidirectional(LSTM(units=n_past, return_sequences=True, input_shape = x_train.shape)))
regressor.add(Dropout(dropout))

regressor.add(LSTM(units = n_past, return_sequences = True))
regressor.add(Dropout(dropout))

regressor.add(LSTM(units = n_past, return_sequences = True))
regressor.add(Dropout(dropout))

regressor.add(LSTM(units = n_past, return_sequences = True))
regressor.add(Dropout(dropout))
regressor.add(Dense(units = n_future, activation = 'linear'))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['acc'])

regressor.fit(x_train[0], y_train, epochs = 100, batch_size = 3)

(1, 11, 235, 1538)
Epoch 1/100
4/4 [==============================] - 6s 164ms/step - loss: 0.0504 - acc: 0.9495
Epoch 2/100
4/4 [==============================] - 1s 165ms/step - loss: 0.0501 - acc: 0.9491
Epoch 3/100
4/4 [==============================] - 1s 166ms/step - loss: 0.0496 - acc: 0.9495
Epoch 4/100
4/4 [==============================] - 1s 162ms/step - loss: 0.0492 - acc: 0.9495
Epoch 5/100
4/4 [==============================] - 1s 162ms/step - loss: 0.0491 - acc: 0.9495
Epoch 6/100
4/4 [==============================] - 1s 163ms/step - loss: 0.0489 - acc: 0.9495
Epoch 7/100
4/4 [==============================] - 1s 163ms/step - loss: 0.0487 - acc: 0.9495
Epoch 8/100
4/4 [==============================] - 1s 161ms/step - loss: 0.0486 - acc: 0.9495
Epoch 9/100
4/4 [==============================] - 1s 162ms/step - loss: 0.0485 - acc: 0.9495
Epoch 10/100
4/4 [==============================] - 1s 162ms/step - loss: 0.0483 - acc: 0.9495
Epoch 11/100
4/4 [======================

4/4 [==============================] - 1s 164ms/step - loss: 0.0479 - acc: 0.9495
Epoch 88/100
4/4 [==============================] - 1s 165ms/step - loss: 0.0479 - acc: 0.9495
Epoch 89/100
4/4 [==============================] - 1s 168ms/step - loss: 0.0479 - acc: 0.9495
Epoch 90/100
4/4 [==============================] - 1s 169ms/step - loss: 0.0479 - acc: 0.9495
Epoch 91/100
4/4 [==============================] - 1s 169ms/step - loss: 0.0479 - acc: 0.9495
Epoch 92/100
4/4 [==============================] - 1s 164ms/step - loss: 0.0479 - acc: 0.9495
Epoch 93/100
4/4 [==============================] - 1s 166ms/step - loss: 0.0480 - acc: 0.9495
Epoch 94/100
4/4 [==============================] - 1s 167ms/step - loss: 0.0479 - acc: 0.9495
Epoch 95/100
4/4 [==============================] - 1s 167ms/step - loss: 0.0479 - acc: 0.9495
Epoch 96/100
4/4 [==============================] - 1s 164ms/step - loss: 0.0479 - acc: 0.9495
Epoch 97/100
4/4 [==============================] - 1s 165ms/st

In [136]:
#from sklearn.preprocessing import MinMaxScaler
#sc = MinMaxScaler(feature_range=(0,1))
print(x_train.shape)

#training_set_scaled = sc.fit_transform(x_train)

x_train_x = x_train[3]

print(x_train_x.shape)

predict_train = regressor.predict(x_train_x)


predict_train = np.reshape(predict_train, (7, 235))
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(predict_train)


predicted_fires = sc.inverse_transform(predict_train)




print(predicted_fires)


print(x_train[0][5].size)





predicted_fires.shape

(5, 7, 235, 1538)
(7, 235, 1538)
[[0.01728484 0.01779686 0.01707474 ... 0.02160396 0.02203205 0.02243012]
 [0.01728484 0.01779686 0.01707472 ... 0.02158587 0.02201759 0.02241862]
 [0.0172847  0.01779448 0.01706103 ... 0.02160396 0.02203205 0.02243012]
 ...
 [0.01728484 0.01779686 0.01707474 ... 0.02160396 0.02203205 0.02243012]
 [0.01728484 0.01779686 0.01707472 ... 0.02158587 0.02201759 0.02241862]
 [0.0172847  0.01779448 0.01706103 ... 0.02160396 0.02203205 0.02243012]]
361430


(7, 235)

190681.0


In [ ]:
pip install sc-tool